## Pandas `rolling().apply()` 

All previous notebooks discussed `apply()` in the context of non-timeseries data. 
When working with timeseries data, we often want to calculate simple rolling 
statistics such as rolling sums (also called cumulative sum), averages, medians, 
and standard deviations. That's when we use `rolling().apply()`. Let me 
demonstrate with DEX volume data from DeFiLlama.

In [1]:
import pandas as pd
import numpy as np
from defillama2 import DefiLlama
from typing import Union

In [2]:
def equal(
    a: Union[pd.DataFrame, pd.Series, np.ndarray], 
    b: Union[pd.DataFrame, pd.Series, np.ndarray]):
    """ 
    Check if the corresponding values of two data frames or series or numpy arrays are the same.
    """
    return (abs(a - b) > 1e-8).sum().sum() == 0 # 0 means same values

In [3]:
obj = DefiLlama() # create a DefiLlama instance
dd = obj.get_dexes_volumes()
dd.keys()

dict_keys(['volume_overall', 'volume_by_dex', 'volume_by_dex_by_chain_24h', 'daily_volume', 'daily_volume_by_dex'])

In [4]:
df = dd['daily_volume'] # focus on daily volumes
df.head()

,volume
date,
2019-10-11 00:00:00+00:00,9.435028e+05
2019-10-12 00:00:00+00:00,6.372715e+05
2019-10-13 00:00:00+00:00,5.939759e+05
2019-10-14 00:00:00+00:00,1.160122e+06
2019-10-15 00:00:00+00:00,1.118992e+06


### Calculate Rolling Means, Medians, Standard Deviations, and Cumulative Sum

In [5]:
# 7-day rolling means
rolling_means_7d = df['volume'].rolling(7).apply(np.mean)
# 7-day rolling median
rolling_meds_7d = df['volume'].rolling(7).apply(np.median)
# 7-day rolling stds
rolling_stds_7d = df['volume'].rolling(7).apply(np.std)
# 1-day rolling sums
rolling_sums_1d = df['volume'].rolling(1).apply(np.sum)

For these simple rolling statistics, there are designated functions that perform better than the `apply()` versions above.

In [6]:
# 7-day rolling means
rolling_means_7d_v2 = df['volume'].rolling(7).mean()
# 7-day rolling median
rolling_meds_7d_v2 = df['volume'].rolling(7).median()
# 7-day rolling stds
rolling_stds_7d_v2 = df['volume'].rolling(7).std()
# 1-day rolling sums
rolling_sums_1d_v2 = df['volume'].cumsum()

In [7]:
assert(equal(rolling_means_7d, rolling_means_7d_v2))
assert(equal(rolling_meds_7d, rolling_meds_7d_v2))
assert(equal(rolling_stds_7d, rolling_stds_7d_v2))
assert(equal(rolling_sums_1d, rolling_sums_1d_v2))

AssertionError: 

The above code will also work on a data frame of all numerical columns. 
Let me show you. First, let me prepare a data frame of all three versions of 
Uniswap's daily volumes since 01 Jan 2023.

In [ ]:
df = dd['daily_volume_by_dex']
dexes = ['Uniswap V1', 'Uniswap V2', 'Uniswap V3']
subdf = df.loc['2023-01-01':, dexes]
subdf.head()

,Uniswap V1,Uniswap V2,Uniswap V3
date,,,
2023-01-01 00:00:00+00:00,63368.031010,3.175149e+07,3.004782e+08
2023-01-02 00:00:00+00:00,20800.457170,3.790035e+07,4.418249e+08
2023-01-03 00:00:00+00:00,8090.465679,3.136464e+07,3.224987e+08
2023-01-04 00:00:00+00:00,45177.833705,4.748669e+07,8.069168e+08
2023-01-05 00:00:00+00:00,72497.322509,8.884805e+07,4.424645e+08


I can then copy and paste the above `rolling().apply()` code to apply to this new frame.

In [ ]:
# 7-day rolling means
rolling_means_7d = subdf.rolling(7).apply(np.mean)
# 7-day rolling median
rolling_meds_7d = subdf.rolling(7).apply(np.median)
# 7-day rolling stds
rolling_stds_7d = subdf.rolling(7).apply(np.std)
# 1-day rolling sums
rolling_sums_1d = subdf.rolling(1).apply(np.sum)

And of course, the better versions also work!

In [ ]:
# 7-day rolling means
rolling_means_7d_v2 = subdf.rolling(7).mean()
# 7-day rolling median
rolling_meds_7d_v2 = subdf.rolling(7).median()
# 7-day rolling stds
rolling_stds_7d_v2 = subdf.rolling(7).std()
# 1-day rolling sums
rolling_sums_1d_v2 = subdf.rolling(1).sum()

In [ ]:
assert(equal(rolling_means_7d, rolling_means_7d_v2))
assert(equal(rolling_meds_7d, rolling_meds_7d_v2))
assert(equal(rolling_stds_7d, rolling_stds_7d_v2))
assert(equal(rolling_sums_1d, rolling_sums_1d_v2))

And there's a more versatile function called `agg()` that allows us to calculate 
different and/or multiple rolling statistics for different columns. 

In [ ]:
subdf.rolling(7, min_periods=7).agg(
    {'Uniswap V1': ['median', 'mean'], 
     'Uniswap V2': ['mean', 'std'],
     'Uniswap V3': ['sum'],
})

Uniswap V1                  Uniswap V2  \
                                 median          mean          mean   
date                                                                  
2023-01-01 00:00:00+00:00           NaN           NaN           NaN   
2023-01-02 00:00:00+00:00           NaN           NaN           NaN   
2023-01-03 00:00:00+00:00           NaN           NaN           NaN   
2023-01-04 00:00:00+00:00           NaN           NaN           NaN   
2023-01-05 00:00:00+00:00           NaN           NaN           NaN   
...                                 ...           ...           ...   
2023-07-22 00:00:00+00:00  34806.931380  35466.287339  1.292651e+08   
2023-07-23 00:00:00+00:00  30228.619734  29588.990736  1.331265e+08   
2023-07-24 00:00:00+00:00  30228.619734  29098.222843  1.441589e+08   
2023-07-25 00:00:00+00:00  30228.619734  29723.699323  1.500543e+08   
2023-07-26 00:00:00+00:00  30228.619734  29391.135849  1.397862e+08   

                                           Uniswap V3  
                                    std           sum  
date                                                   
2023-01-01 00:00:00+00:00           NaN           NaN  
2023-01-02 00:00:00+00:00           NaN           NaN  
2023-01-03 00:00:00+00:00           NaN           NaN  
2023-01-04 00:00:00+00:00           NaN           NaN  
2023-01-05 00:00:00+00:00           NaN           NaN  
...                                 ...           ...  
2023-07-22 00:00:00+00:00  2.458436e+07  4.810233e+09  
2023-07-23 00:00:00+00:00  2.429102e+07  4.727569e+09  
2023-07-24 00:00:00+00:00  3.929839e+07  4.456881e+09  
2023-07-25 00:00:00+00:00  3.175472e+07  4.075148e+09  
2023-07-26 00:00:00+00:00  3.645313e+07  4.102507e+09  

[207 rows x 5 columns]

Beyond simple rolling statistics, there are no readily available functions in 
Pandas that we can use to compute complex statistics. For example, say we want to 
calculate the 30-day rolling auto-correlations of the daily volumes for each 
Uniswap version. We can't simply use `subdf.rolling(30).autocorr()`. Instead, 
we can use `apply(lambda ser: ser.autocorr())`.

In [ ]:
# # try it. It will throw an error
# subdf.rolling(30).autocorr()

In [ ]:
subdf.rolling(30).apply(lambda ser: ser.autocorr())

,Uniswap V1,Uniswap V2,Uniswap V3
date,,,
2023-01-01 00:00:00+00:00,NaN,NaN,NaN
2023-01-02 00:00:00+00:00,NaN,NaN,NaN
2023-01-03 00:00:00+00:00,NaN,NaN,NaN
2023-01-04 00:00:00+00:00,NaN,NaN,NaN
2023-01-05 00:00:00+00:00,NaN,NaN,NaN
...,...,...,...
2023-07-22 00:00:00+00:00,-0.068213,0.373489,0.047972
2023-07-23 00:00:00+00:00,-0.073116,0.396239,0.118452
2023-07-24 00:00:00+00:00,-0.082624,0.444259,0.128597


### Summary

- `rolling().apply(func)` is very handy when calculating rolling statistics 
  on numerical columns of a data frame (or a numerical series). 
- when calculating simple rolling statistics such as sum, mean, median, and 
  standard deviation, drop `apply()` and call the statistical functions directly.
- `rolling().agg()` allows calculation of different and/or multiple simple 
  rolling statistics for different columns. 

There are two drawbacks of `rolling().apply()`:
1. it is slow and memory-inefficient.
2. it cannot apply a function that takes two columns (series, ndarrays) as input. 
   For example, in this [dashboard](https://coindataschool-husdlci-main-pfjljd.streamlit.app/), 
   I calculate and visualize 30-day rolling correlations between BTC price and 
   Hayes USD liquidity condition index. This can't be done by `rolling().apply()`.

I'll dive deep into these two drawbacks and provide solutions in the upcoming 
notebooks. Star and watch the repo to stay informed.

### Good Read

- [All my notebooks on pandas `apply()`](https://coindataschool.substack.com/p/pandas-apply)